In [7]:
import requests
from bs4 import BeautifulSoup
import re

In [8]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

200


In [9]:
if res.ok:
    html = res.text
    #html 전체 소스보기
    #print(html)
    soup = BeautifulSoup(html, 'html.parser')
    #html parsing
    #print(soup)  

In [10]:

#parsing한 html중 곡에대한 정보를 담음 태그만 조회!!
a_tags = soup.select("div#tb_list tr a[href*='playSong']")
print(len(soup.select("div#tb_list tr a[href*='playSong']")))  
#top100이니까 100이 나와야한다.
#print(a_tags)

100


In [11]:
# playSong이 있는 <a></a>를 enumerate하여 dict형식으로 만들어준다.
#Top 100곡의 정보를 저장할 list 선언
song_list = []
for idx, a_tag in enumerate(a_tags,1):
    #노래 1곡의 정보를 저장할 dict 선언 
    song_dict = {}
    #노래 제목 갖고오기 <a href="">노래제목</a>
    song_title = a_tag.text
    song_dict['song_title'] = song_title
    
    #print(song_dict)
    
    #a태그의 href 속성의 값을 추출하기 javascript:melon.play.playSong('1000002721',34535898);
    href_value = a_tag['href']

    #Song ID를 찾기 위한 정규표현식
    matched = re.search(r'(\d+)\);', href_value)
    #print(matched)
    if matched:
        song_id = matched.group(1) # group(0) : 34535898);  group(1) : 34535898
        song_dict['song_id'] = song_id
        #print(song_id)
        
        song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}' #곡정보의 url
        song_dict['song_detail_url'] = song_detail_url
        song_list.append(song_dict)

print(len(song_list))    
print(song_list[:3])

100
[{'song_title': '취중고백', 'song_id': '34431086', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34431086'}, {'song_title': '호랑수월가', 'song_id': '34535898', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34535898'}, {'song_title': '사랑은 늘 도망가', 'song_id': '34061322', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34061322'}]


In [12]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)


200


In [55]:
song_detail_list = []
for idx, song in enumerate(song_list[:3],1):
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    
    res = requests.get(song_detail_url, headers=req_header_dict)
    #print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(idx,song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        #노래 1곡의 상세정보에 '곡명'을 찾아 주었다. 
        #다음으로 상세정보 '가수'를 찾는다
        singer = soup.select("a[href*='goArtistDetail'] span")
        #print(singer)
        if singer:
            song_detail_dict['가수'] = singer[0].text # singer리스트에서 0번째 순서에 해당하는것을 text로 뽑으면 이름이다.
        #print(singer[0].text)
        
        #다음으로 div.meta에서 dd로 끝나는 값들을 넣어준다
        song_di = soup.select("div.meta dd")
        #print(song_di)
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text
            song_detail_dict['FLAC'] = song_di[3].text
        
        song_id = song['song_id'] #song_id = matched.group(1) 갖고 와서 곡별 고유 id를 이용하자
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        
        
        if like_res.ok:
           song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
           #{'contsLike': [{'CONTSID': 34061322, 'LIKEYN': 'N', 'SUMMCNT': 124335}], 'httpDomain': 'http://www.melon.com', 'httpsDomain': 'https://www.melon.com', 'staticDomain': 'https://static.melon.co.kr'}
            #좋아요 수 추가 완료
        
        song_lyric = soup.select("div#d_video_summary")
        if song_lyric:
            song_detail_dict['가사'] = song_lyric[0].text
            pattern = re.compile(r'[\r\n\t]')
            # \r\n\t 특수문자를 ''(empty string)으로 대체(substitute)해라
            lyric = pattern.sub('', song_detail_dict['가사'].strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
    
        
        song_detail_list.append(song_detail_dict)   
print(song_detail_list)

[{'곡명': '취중고백', '가수': '김민석 (멜로망스)', '앨범': '취중고백', '발매일': '2021.12.19', '장르': '발라드', 'FLAC': 'Flac 16bit', '좋아요': 69231, '가사': '\r\n\t\t\t\t\t\t\t뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서 오랜만에술을 좀 했는데자꾸만 니 얼굴 떠올라무작정 달려왔어이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해눈물이 날만큼 원하고 있어정말로 몰랐니가끔 전화해 장난치듯주말엔 뭐할거냐며너의 관심 끌던 나를그리고 한번씩 누나 주려 샀는데너 그냥 준다고생색 낸 선물도 너 때문에 산거야이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해진심이야 믿어줘갑자기 이런 말 놀랐다면 미안해부담이 되는게 당연해이해해 널하지만 내 고백도 이해해 주겠니 oh지금 당장 대답하진마나와 일주일만 사귀어줄래후회없이 잘 해주고 싶은데그 후에도 니가 싫다면나 그때 포기할게귀찮게 안할게 혼자 아플게진심이야 너를 사랑하고 있어\n'}, {'곡명': '호랑수월가', '가수': '탑현', '앨범': '호랑수월가', '발매일': '2022.01.15', '장르': '발라드', 'FLAC': 'Flac 16bit', '좋아요': 36042, '가사': '\r\n\t\t\t\t\t\t\t흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가는 강아지처럼온 산에 풍물 막을 내리네바람은 지친 끝에 밤에 몸을 뉘이네별빛은 아뜩하니 은하수를 내리네차가운 밤하늘에 세상이 젖어 가네그리워 홀로 타령을 하자흘러가라 사랑사랑아덧없이 피고 떨어지는 꽃송아애닯구나 가락가락아눈물에 떨어진 별을 헤네푸른 달아 오랜 고운 내 달아비친 내 손에 내려다오은색 소매 내 곁에 두른 채로한 번만 타는 입을 축여다오푸른 달아 다시 없을 내 달아뻗은 손끝에 닿아다오